In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
df = pd.read_csv('epinions.txt',sep='\t')


In [2]:
G = nx.DiGraph()
for index,rows in df.iterrows():
    u = rows['FromNodeId']
    v = rows['ToNodeId']
    s = rows['Sign']
    G.add_edge(u,v,sign=s)

In [3]:
in_pos = {}
in_neg = {}
in_pos_count ={}
in_neg_count = {}



for nodes in G.nodes():
    l = list(G.in_edges(nodes,data=True))
    pos=[]
    neg=[]
    
    for items in l:
        if items[2]['sign'] == 1:
            pos.append(items[0])
            
        else:
            neg.append(items[0])
    in_pos[nodes] = pos
    in_neg[nodes] = neg
    in_pos_count[nodes] = len(pos)
    in_neg_count[nodes] = len(neg)



out_pos = {}
out_neg = {}
out_pos_count= {}
out_neg_count = {}
for nodes in G.nodes():
    pos = []
    neg = []
    l = dict(G[nodes])
    for keys in l:
        if(l[keys]['sign'] == 1):
            pos.append(keys)
        else:
            neg.append(keys)
    out_pos[nodes] = pos
    out_pos_count[nodes] = len(pos)
    out_neg[nodes] = neg
    out_neg_count[nodes] = len(neg)

nx.set_node_attributes(G,in_pos,'in_pos')
nx.set_node_attributes(G,in_neg,'in_neg')
nx.set_node_attributes(G,out_pos,'out_pos')
nx.set_node_attributes(G,out_neg,'out_neg')
nx.set_node_attributes(G,out_pos_count,'out_pos_count')
nx.set_node_attributes(G,out_neg_count,'out_neg_count')
nx.set_node_attributes(G,in_pos_count,'in_pos_count')
nx.set_node_attributes(G,in_neg_count,'in_neg_count')


In [4]:
avg_in_pos = 0;
length = 0;
for item in in_pos_count:
    avg_in_pos += in_pos_count[item]
    length+=1
avg_in_pos = avg_in_pos/length
avg_in_pos

5.443964863306733

In [5]:
avg_in_neg = 0;
length = 0;
for item in in_neg_count:
    avg_in_neg += in_neg_count[item]
    length+=1
    

avg_in_neg = avg_in_neg/length
avg_in_neg

0.9383818308705283

In [6]:
Nodes = G.nodes(data=True)

In [7]:
optimism_lower_than_avg = {}
for node in Nodes:
    oli=[]
    for item in node[1]['out_pos']:
        if in_neg_count[item]>avg_in_neg:
            oli.append(item)
    optimism_lower_than_avg[node[0]] = oli

In [8]:
nx.set_node_attributes(G, optimism_lower_than_avg , 'ol_array')

In [9]:
OH = {}
for node in Nodes:
    oli=[]
    for item in node[1]['ol_array']:
        if in_pos_count[item]<avg_in_pos:
            oli.append(item)
    OH[node[0]] = oli
nx.set_node_attributes(G, OH , 'oh_array')

In [10]:
optimism = {} 
for node in OH:
    X = len(OH[node])
    Y = len(optimism_lower_than_avg[node])
    if Y >0:
        optimism[node] = X/Y
    else:
        optimism[node] = 0

In [11]:
pessimism_h = {}
for node in Nodes:
    phi=[]
    for item in node[1]['out_neg']:
        if in_pos_count[item]>avg_in_pos:
            phi.append(item)
    pessimism_h[node[0]] = phi
nx.set_node_attributes(G, pessimism_h , 'ph_array')

In [12]:
pl = {}
for node in Nodes:
    oli=[]
    for item in node[1]['ph_array']:
        if in_neg_count[item]>avg_in_neg:
            oli.append(item)
    pl[node[0]] = oli

pl

{0: [],
 1: [],
 128552: [],
 2: [],
 3: [],
 4: [5, 155, 1509, 10876, 48703],
 5: [264,
  410,
  532,
  744,
  819,
  865,
  1003,
  1118,
  1262,
  1268,
  1488,
  1491,
  1493,
  1512,
  1578,
  1611,
  1622,
  1652,
  1713,
  1741,
  1750,
  1818,
  1819,
  1829,
  1831,
  1834,
  1853,
  1882,
  1894,
  1937,
  2183,
  2243,
  2278,
  2358,
  2363,
  2547,
  2723,
  2755,
  2995,
  3144,
  3172,
  3177,
  3396,
  3798,
  3838,
  3876,
  4173,
  4436,
  4446,
  4494,
  4561,
  4590,
  4774,
  4887,
  5603,
  5641,
  5729,
  5819,
  6176,
  6296,
  6328,
  6392,
  6462,
  6834,
  7062,
  7821,
  8684,
  9310,
  9343,
  9665,
  9698,
  9825,
  10428,
  11086,
  11205,
  11675,
  11751,
  12359,
  13539,
  13683,
  14124,
  14416,
  15654,
  15979,
  16697,
  16751,
  16786,
  16992,
  16995,
  17020,
  17042,
  17059,
  17360,
  17391,
  17789,
  18148,
  20109,
  20383,
  20597,
  21263,
  21286,
  21643,
  23559,
  24138,
  24881,
  25689,
  26084,
  26099,
  26167,
  26168,
  2633

In [13]:
pessimism = {} 
for node in pl:
    
    X = len(pl[node])
    Y = len(pessimism_h[node])
    if Y >0:
        pessimism[node] = X/Y
    else:
        pessimism[node] = 0

In [14]:
 nx.set_node_attributes(G, pessimism, 'pessimism')
 nx.set_node_attributes(G, optimism, 'optimism')

In [15]:
Edges = G.edges()


In [18]:
pesimism_u = {}
pesimism_v = {}
optimism_u = {}
optimism_v = {}
positivity_u = {}
positivity_v = {}
negativity_u = {}
negativity_v = {}

for edge in Edges:
    u = edge[0]
    v = edge[1]
    p1 = Nodes[u]['pessimism']
    p2 = Nodes[v]['pessimism']
    o1 = Nodes[v]['optimism']
    o2 = Nodes[v]['optimism']
    pos1 = len(Nodes[u]['ol_array'])
    pos2 = len(Nodes[v]['ol_array'])
    neg1 = len(Nodes[u]['ph_array'])
    neg2 = len(Nodes[v]['ph_array'])
    pesimism_u[edge] = p1
    pesimism_v[edge] = p2
    optimism_u[edge] = o1
    optimism_v[edge] = o2
    positivity_u[edge]= pos1
    positivity_v[edge] = pos2
    negativity_u[edge] = neg1
    negativity_v[edge] = neg2
    

In [19]:
nx.set_edge_attributes(G,  positivity_u, 'positivity_u')
nx.set_edge_attributes(G,  positivity_v, 'positivity_v')
nx.set_edge_attributes(G,  negativity_u, 'negativity_u')
nx.set_edge_attributes(G,  negativity_v, 'negativity_v')
nx.set_edge_attributes(G,  pesimism_u, 'pesimism_u')
nx.set_edge_attributes(G,  pesimism_v, 'pesimism_v')
nx.set_edge_attributes(G,  optimism_u, 'optimism_u')
nx.set_edge_attributes(G,  optimism_v, 'optimism_v')

In [20]:
df = pd.DataFrame(columns= ['edge', 'positivity_u', 'positivity_v', "negativity_u", 'negativity_v', 'pesimism_u', 'pesimism_v', 'optimism_u', 'optimism_v', 'sign' ])

In [21]:
positivity_u = nx.get_edge_attributes(G, 'positivity_u')
positivity_v = nx.get_edge_attributes(G, 'positivity_v')
negativity_u = nx.get_edge_attributes(G, 'negativity_u')
negativity_v = nx.get_edge_attributes(G, 'negativity_v')
pesimism_u = nx.get_edge_attributes(G, 'pesimism_u')
pesimism_v = nx.get_edge_attributes(G, 'pesimism_v')
optimism_u = nx.get_edge_attributes(G, 'optimism_u')
optimism_v = nx.get_edge_attributes(G, 'optimism_v')
edge = positivity_u.keys()
sign = nx.get_edge_attributes(G, 'sign')

In [22]:
df['sign'] = sign.values()
df['positivity_u'] = positivity_u.values()
df['positivity_v'] = positivity_v.values()
df['negativity_u'] = negativity_u.values()
df['negativity_v'] = negativity_v.values()
df['pesimism_u'] = pesimism_u.values()
df['pesimism_v'] = pesimism_v.values()
df['optimism_u'] = optimism_u.values()
df['optimism_v'] = optimism_v.values()
df['edge'] = positivity_u.keys()


In [23]:
df.to_csv('emotional_informations_epinions.csv')
df

,edge,positivity_u,positivity_v,negativity_u,negativity_v,pesimism_u,pesimism_v,optimism_u,optimism_v,sign
0,"(0, 1)",0,0,0,0,0.0,0.0,0.000000,0.000000,-1
1,"(1, 128552)",0,0,0,0,0.0,0.0,0.000000,0.000000,-1
2,"(2, 3)",0,0,0,0,0.0,0.0,0.000000,0.000000,1
3,"(4, 5)",4,435,5,174,1.0,1.0,0.006897,0.006897,-1
4,"(4, 155)",4,399,5,9,1.0,1.0,0.002506,0.002506,-1
5,"(4, 558)",4,17,5,0,1.0,0.0,0.000000,0.000000,1
6,"(4, 1509)",4,49,5,114,1.0,1.0,0.000000,0.000000,-1
7,"(4, 2282)",4,154,5,46,1.0,1.0,0.006494,0.006494,1
8,"(4, 2984)",4,28,5,9,1.0,1.0,0.000000,0.000000,1
9,"(4, 7263)",4,3,5,0,1.0,0.0,0.000000,0.000000,1


In [24]:
from sklearn.model_selection import train_test_split
X = df.drop(['edge','sign'],axis=1)
y = df['sign']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
lr = LogisticRegression()
lr.fit(X_train,y_train)
pred = lr.predict(X_test)
lr.score(X_test,y_test)

0.900930025256277

AttributeError: 'DataFrame' object has no attribute 'isNa'

b###### 